En este práctico vamos a aplicar PCA a un dataset de composiciones química de 3 diferentes vinos.
Analizaremos qué componentes son importantes, osea, aquellos que contiene la mayor varianza.

In [ ]:
# Dataset de la composición química de 3 diferentes tipos de vinos.
# El target es el tipo de vino.

# https://archive.ics.uci.edu/ml/datasets/Wine

import pandas as pd
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)

df_wine.columns = ['Class label', 
                   'Alcohol',
                   'Malic acid', 
                   'Ash',
                   'Alcalinity of ash', 
                   'Magnesium',
                   'Total phenols', 
                   'Flavanoids',
                   'Nonflavanoid phenols',
                   'Proanthocyanins',
                   'Color intensity', 
                   'Hue',
                   'OD280/OD315 of diluted wines',
                   'Proline']

In [ ]:
df_wine.describe()

In [ ]:
df_wine.shape

### Separo el dataset en train y test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_wine.iloc[:, 1:].values
y = df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

sc = StandardScaler()

X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

### Matriz de covarianza, vectores y valores propios

In [ ]:
import numpy as np

cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eigh(cov_mat)
print('\nEigenvalues \n%s' % eigen_vals)

### Muestro la variancia acumulada

In [ ]:
# En esta grafica podemos ver que el primer componente contiene el 40% de la varianza
# mientras que los dos primeros componentes poseen casi el 60% de la varianza.

%matplotlib notebook

tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

import matplotlib.pyplot as plt

plt.bar(range(1,14), var_exp, alpha=0.5, align='center',label='individual explained variance')
plt.step(range(1,14), cum_var_exp, where='mid', label='cumulative explained variance')

plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show()

### Comprimimos los features a 2 dimensiones para poder visualizar el dataset

In [ ]:
# Ordenamos los vectores propios en orden decreciente.

eigen_pairs =[(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]

eigen_pairs.sort(reverse=True)

In [ ]:
# Nos quedamos con los dos primeros vectores propios, los que tienen el valor absoluto más grande.
# Generamos una matriz de proyección con los dos vectores propios.

w = np.hstack((eigen_pairs[0][1][:, np.newaxis], eigen_pairs[1][1][:, np.newaxis]))
print('Matrix W:\n',w)

In [ ]:
# Multiplicando x por la matrix W tendremos la proyección de la instancia
# x en el nuevo sub-espacio de dos dimensiones.

X_train_std[0].dot(w)

In [ ]:
# Proyectamos todo el dataset en el sub-espacio de 2 dimensiones.

X_train_pca = X_train_std.dot(w)

In [ ]:
# Por ultimo  graficamos cada instancia en un plano.
# Podemos observar que con solo dos features poseemos suficiente
# información para clusterizar los tipos de vino.
# Importante: Tener en cuenta que PCA es un algoritmo no supervisado,
# significa que nunca tuvo en cuenta los target del tipo de vino,
# en esta gráfica se coloreó las instancias para poder visualizar el restultado.

%matplotlib notebook

colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']

for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_pca[y_train==l, 0], X_train_pca[y_train==l, 1], c=c, label=l, marker=m)
    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.show()